## Implementation

In [13]:
from IOHMM import IOHMM_model
import numpy as np, pandas as pd
import torch

In [2]:
data = pd.read_csv("data/Financial-Data/stocks/TSLA.csv")
data.head()
data = data.dropna()
input_data = data[['Open', 'High', 'Low', 'Close']][:-1]


# close price of the next day
output_data = data['Close'][1:]
# to start from 0 index
output_data.index = range(len(output_data))

output_data.name = 'Close-next-day'

data = pd.concat([input_data, output_data], axis=1)

data.head()
print(len(data))


input = torch.tensor(np.array(data[['Open', 'High', 'Low', 'Close']]), dtype=torch.float32)
output = torch.tensor(np.array(data['Close-next-day']), dtype=torch.float32)

print(output[0:5])
print(input[0:5])
torch.cat((torch.tensor([1.0]), output))


3259
tensor([1.5887, 1.4640, 1.2800, 1.0740, 1.0533])
tensor([[1.2667, 1.6667, 1.1693, 1.5927],
        [1.7193, 2.0280, 1.5533, 1.5887],
        [1.6667, 1.7280, 1.3513, 1.4640],
        [1.5333, 1.5400, 1.2473, 1.2800],
        [1.3333, 1.3333, 1.0553, 1.0740]])


tensor([  1.0000,   1.5887,   1.4640,  ..., 224.5700, 234.8600, 244.4000])

In [3]:
data = pd.read_csv("data/Financial-Data/stocks/TSLA.csv")
data.head()
data = data.dropna()
input_data = data[['Open', 'High', 'Low', 'Close']][:-1]


# close price of the next day
output_data = data['Close'][1:]
# to start from 0 index
output_data.index = range(len(output_data))

output_data.name = 'Close-next-day'

data = pd.concat([input_data, output_data], axis=1)

data.head()
print(len(data))


input = torch.tensor(np.array(data[['Open', 'High', 'Low', 'Close']]), dtype=torch.float32)
output = torch.tensor(np.array(data['Close-next-day']), dtype=torch.float32)

print(output[0:5])
print(input[0:5])
torch.cat((torch.tensor([1.0]), output))


3259
tensor([1.5887, 1.4640, 1.2800, 1.0740, 1.0533])
tensor([[1.2667, 1.6667, 1.1693, 1.5927],
        [1.7193, 2.0280, 1.5533, 1.5887],
        [1.6667, 1.7280, 1.3513, 1.4640],
        [1.5333, 1.5400, 1.2473, 1.2800],
        [1.3333, 1.3333, 1.0553, 1.0740]])


tensor([  1.0000,   1.5887,   1.4640,  ..., 224.5700, 234.8600, 244.4000])

# Example 1

## Set up a simple model manully

In [4]:
IOHMM = IOHMM_model(num_states=2, inputs=input[:10], outputs=output[:10], max_iter=1000, tol=1e-6)
print(IOHMM.initial_pi)
print(IOHMM.transition_matrix)
print(IOHMM.emission_matrix)
print(IOHMM.sd)

Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True)
Parameter containing:
tensor([[[-0.2661, -1.9592,  0.1232,  0.0702, -0.8170],
         [-0.3367,  1.6466,  1.4688, -3.0883,  1.3104]],

        [[ 0.4049,  0.1141,  0.4214,  0.0888, -0.1736],
         [-1.9460, -0.7568, -0.5346, -1.2838,  0.2673]]], requires_grad=True)
Parameter containing:
tensor([[-0.7389, -2.3656,  0.6441,  1.1685,  1.4863],
        [ 0.2303,  0.0119,  0.0257, -0.0583,  0.0672]], requires_grad=True)
Parameter containing:
tensor([1., 1.], requires_grad=True)


In [5]:
print(IOHMM._forward())
torch.sum(IOHMM._forward(), dim=1)

tensor([[0.6598, 0.3402],
        [0.5850, 0.4150],
        [0.4672, 0.5328],
        [0.4269, 0.5731],
        [0.3773, 0.6227],
        [0.4614, 0.5386],
        [0.5275, 0.4725],
        [0.4988, 0.5012],
        [0.4866, 0.5134],
        [0.5365, 0.4635]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [6]:
print(IOHMM._backward())
torch.sum(IOHMM._backward(), dim=1)

tensor([[0.5850, 0.4150],
        [0.4672, 0.5328],
        [0.4269, 0.5731],
        [0.3773, 0.6227],
        [0.4614, 0.5386],
        [0.5275, 0.4725],
        [0.4988, 0.5012],
        [0.4866, 0.5134],
        [0.5365, 0.4635],
        [0.6140, 0.3860]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [7]:
print(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()))
torch.sum(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()), dim=1)

tensor([[0.7322, 0.2678],
        [0.5527, 0.4473],
        [0.3951, 0.6049],
        [0.3110, 0.6890],
        [0.3418, 0.6582],
        [0.4889, 0.5111],
        [0.5263, 0.4737],
        [0.4854, 0.5146],
        [0.5231, 0.4769],
        [0.6480, 0.3520]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [8]:
print(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()))
a = torch.sum(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()), axis=1)
torch.sum(a, axis=1)

tensor([[[0.3098, 0.1902],
         [0.3098, 0.1902]],

        [[0.3148, 0.1852],
         [0.3148, 0.1852]],

        [[0.2560, 0.2440],
         [0.2560, 0.2440]],

        [[0.1735, 0.3265],
         [0.1735, 0.3265]],

        [[0.1950, 0.3050],
         [0.1950, 0.3050]],

        [[0.2033, 0.2967],
         [0.2033, 0.2967]],

        [[0.2314, 0.2686],
         [0.2314, 0.2686]],

        [[0.2582, 0.2418],
         [0.2582, 0.2418]],

        [[0.2690, 0.2310],
         [0.2690, 0.2310]],

        [[0.3016, 0.1984],
         [0.3016, 0.1984]]])


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [9]:
#IOHMM._baum_welch()

## See the training results

In [10]:
print("Initial pi")
print(IOHMM.initial_pi) # to normalize
print("Transition matrix")
print(IOHMM.transition_matrix)
print("Emission matrix")
print(IOHMM.emission_matrix)
print("Sd")
print(IOHMM.sd)

Initial pi
Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True)
Transition matrix
Parameter containing:
tensor([[[-0.2661, -1.9592,  0.1232,  0.0702, -0.8170],
         [-0.3367,  1.6466,  1.4688, -3.0883,  1.3104]],

        [[ 0.4049,  0.1141,  0.4214,  0.0888, -0.1736],
         [-1.9460, -0.7568, -0.5346, -1.2838,  0.2673]]], requires_grad=True)
Emission matrix
Parameter containing:
tensor([[-0.7389, -2.3656,  0.6441,  1.1685,  1.4863],
        [ 0.2303,  0.0119,  0.0257, -0.0583,  0.0672]], requires_grad=True)
Sd
Parameter containing:
tensor([1., 1.], requires_grad=True)


## Viterbi

In [11]:
#call the viterbi algorithm
IOHMM.viterbi()

tensor([0, 1, 1, 1, 1, 0, 1, 1, 0, 0])